In [2]:

import pandas as pd

# Load survey + orders
survey = pd.read_csv("../data/raw/fruvibe_survey.csv")
orders = pd.read_csv("../data/raw/fruvibe_orders.csv")

# Quick check
print("Survey shape:", survey.shape)
print("Orders shape:", orders.shape)

survey.head(), orders.head()


FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/fruvibe_survey.csv'

In [3]:
import pandas as pd

# Load survey + orders
survey = pd.read_csv("../data/raw/fruvibe_survey.csv")
orders = pd.read_csv("../data/raw/fruvibe_orders.csv")

# Quick check
print("Survey shape:", survey.shape)
print("Orders shape:", orders.shape)

survey.head(), orders.head()


FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/fruvibe_survey.csv'

In [4]:
import os
print(os.getcwd())  # shows current working directory



C:\Users\nivas\OneDrive\Desktop\fruvube project\Fruvvibe


In [5]:
import pandas as pd

# Load survey + orders
survey = pd.read_csv("data/raw/fruvibe_survey.csv")
orders = pd.read_csv("data/raw/fruvibe_orders.csv")

# Quick check
print("Survey shape:", survey.shape)
print("Orders shape:", orders.shape)

survey.head(), orders.head()


Survey shape: (276, 11)
Orders shape: (890, 13)


(   RespondentID  Age Income_Level PostcodeDistrict             AreaGroup  \
 0             1   47      20k-40k              E14             Docklands   
 1             2   59         <20k              IG3      Ilford-Redbridge   
 2             3   19         <20k               E2  Hackney-BethnalGreen   
 3             4   18      40k-60k               E1             Docklands   
 4             5   29         <20k               E7     Stratford-EastHam   
 
   Preferred_Type Preferred_Plan                 Preferred_Platter  \
 0   Subscription        Essence                               NaN   
 1        OneTime            NaN          Immunity Booster Platter   
 2        OneTime            NaN  Heart-Healthy & Vitality Platter   
 3   Subscription        Everjoy                               NaN   
 4        OneTime            NaN          Immunity Booster Platter   
 
   Subscription_Interest DeliveryWindow_Preference Purchase_Frequency  
 0                   Yes                  

In [6]:
print("Orders shape:", orders.shape)
print("Survey shape:", survey.shape)

print("\nNulls — Orders:\n", orders.isna().sum())
print("\nNulls — Survey:\n", survey.isna().sum())

print("\nDtypes — Orders:\n", orders.dtypes)
print("\nDtypes — Survey:\n", survey.dtypes)

print("\nDuplicates — Orders:", orders.duplicated().sum())
print("Duplicates — Survey:", survey.duplicated().sum())


Orders shape: (890, 13)
Survey shape: (276, 11)

Nulls — Orders:
 OrderID               0
OrderDate             0
PostcodeDistrict      0
AreaGroup             0
OrderType             0
Plan                611
Product               0
UnitPrice             0
Quantity              0
Revenue               0
DeliverySlot          0
OnTime                0
Fulfilled             0
dtype: int64

Nulls — Survey:
 RespondentID                   0
Age                            0
Income_Level                   0
PostcodeDistrict               0
AreaGroup                      0
Preferred_Type                 0
Preferred_Plan               173
Preferred_Platter            103
Subscription_Interest          0
DeliveryWindow_Preference      0
Purchase_Frequency             0
dtype: int64

Dtypes — Orders:
 OrderID               int64
OrderDate            object
PostcodeDistrict     object
AreaGroup            object
OrderType            object
Plan                 object
Product              object


In [7]:
import numpy as np

# strip whitespace on all object columns
for df in (orders, survey):
    for c in df.select_dtypes(include="object").columns:
        df[c] = df[c].astype(str).str.strip()

# parse date
orders["OrderDate"] = pd.to_datetime(orders["OrderDate"], errors="coerce")

# expected categories (safety)
valid_types = {"Subscription","OneTime"}
valid_slots = {"Morning","Evening"}
valid_yesno = {"Yes","No"}

def enforce(series, allowed):
    return np.where(series.isin(allowed), series, np.nan)

orders["OrderType"]    = enforce(orders["OrderType"], valid_types)
orders["DeliverySlot"] = enforce(orders["DeliverySlot"], valid_slots)
orders["OnTime"]       = enforce(orders["OnTime"], valid_yesno)
orders["Fulfilled"]    = enforce(orders["Fulfilled"], valid_yesno)

# drop exact duplicate rows (if any)
orders = orders.drop_duplicates()
survey = survey.drop_duplicates()

orders.head(2), survey.head(2)


C:\Users\nivas\AppData\Local\Temp\ipykernel_6088\3879551339.py:9: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  orders["OrderDate"] = pd.to_datetime(orders["OrderDate"], errors="coerce")


(   OrderID  OrderDate PostcodeDistrict          AreaGroup OrderType Plan  \
 0        1 2025-03-16              IG6   Ilford-Redbridge   OneTime  nan   
 1        2 2025-03-16              E12  Stratford-EastHam   OneTime  nan   
 
                       Product  UnitPrice  Quantity  Revenue DeliverySlot  \
 0    Immunity Booster Platter       6.65         1     6.65      Evening   
 1  Strength & Stamina Platter       6.65         1     6.65      Morning   
 
   OnTime Fulfilled  
 0    Yes       Yes  
 1    Yes       Yes  ,
    RespondentID  Age Income_Level PostcodeDistrict         AreaGroup  \
 0             1   47      20k-40k              E14         Docklands   
 1             2   59         <20k              IG3  Ilford-Redbridge   
 
   Preferred_Type Preferred_Plan         Preferred_Platter  \
 0   Subscription        Essence                       nan   
 1        OneTime            nan  Immunity Booster Platter   
 
   Subscription_Interest DeliveryWindow_Preference Purchas

In [8]:
# Re-parse dates with an explicit format (your orders are YYYY-MM-DD)
orders["OrderDate"] = pd.to_datetime(
    orders["OrderDate"], format="%Y-%m-%d", errors="coerce"
)


In [9]:
import pandas as pd
import numpy as np

def clean_object_columns(df):
    for c in df.columns:
        if df[c].dtype == object:
            # strip while preserving NaNs
            df[c] = df[c].where(df[c].notna(), pd.NA)          # keep missing as NA
            df[c] = df[c].astype("string").str.strip()         # Pandas StringDtype
            # turn accidental text 'nan' / '' back into real missing
            df[c] = df[c].replace({"nan": pd.NA, "NaN": pd.NA, "": pd.NA})

clean_object_columns(orders)
clean_object_columns(survey)


In [10]:
print(orders[["Plan","Product"]].head(3))
print("\nNulls — Orders:\n", orders.isna().sum())
print("\nOrders dtypes:\n", orders.dtypes)


      Plan                     Product
0     <NA>    Immunity Booster Platter
1     <NA>  Strength & Stamina Platter
2  Elevate                     Elevate

Nulls — Orders:
 OrderID               0
OrderDate             0
PostcodeDistrict      0
AreaGroup             0
OrderType             0
Plan                611
Product               0
UnitPrice             0
Quantity              0
Revenue               0
DeliverySlot          0
OnTime                0
Fulfilled             0
dtype: int64

Orders dtypes:
 OrderID                      int64
OrderDate           datetime64[ns]
PostcodeDistrict    string[python]
AreaGroup           string[python]
OrderType           string[python]
Plan                string[python]
Product             string[python]
UnitPrice                  float64
Quantity                     int64
Revenue                    float64
DeliverySlot        string[python]
OnTime              string[python]
Fulfilled           string[python]
dtype: object


In [11]:
# Define pilot date (evening deliveries test)
pilot_start = pd.Timestamp("2025-06-10")

# Orders features
orders["PrePostPilot"] = np.where(
    orders["OrderDate"] < pilot_start, "Pre-Pilot", "Post-Pilot"
)
orders["IsSubscription"] = (orders["OrderType"] == "Subscription").astype(int)
orders["IsEvening"] = (orders["DeliverySlot"] == "Evening").astype(int)
orders["IsOnTime"] = (orders["OnTime"] == "Yes").astype(int)
orders["Month"] = orders["OrderDate"].dt.to_period("M").astype(str)

# Survey features
bins = [18,25,35,45,60]
labels = ["18-25","26-35","36-45","46-60"]
survey["Age_Group"] = pd.cut(
    survey["Age"], bins=bins, labels=labels, include_lowest=True, right=True
)

freq_map = {"Weekly":3, "Bi-Weekly":2, "Monthly":1}
survey["Loyalty_Score"] = survey["Purchase_Frequency"].map(freq_map)
survey["Loyalty_Segment"] = survey["Loyalty_Score"].map({3:"High",2:"Medium",1:"Low"})

orders.head(3), survey.head(3)


(   OrderID  OrderDate PostcodeDistrict          AreaGroup     OrderType  \
 0        1 2025-03-16              IG6   Ilford-Redbridge       OneTime   
 1        2 2025-03-16              E12  Stratford-EastHam       OneTime   
 2        3 2025-03-16              IG2   Ilford-Redbridge  Subscription   
 
       Plan                     Product  UnitPrice  Quantity  Revenue  \
 0     <NA>    Immunity Booster Platter       6.65         1     6.65   
 1     <NA>  Strength & Stamina Platter       6.65         1     6.65   
 2  Elevate                     Elevate      39.75         1    39.75   
 
   DeliverySlot OnTime Fulfilled PrePostPilot  IsSubscription  IsEvening  \
 0      Evening    Yes       Yes    Pre-Pilot               0          1   
 1      Morning    Yes       Yes    Pre-Pilot               0          0   
 2      Morning    Yes       Yes    Pre-Pilot               1          0   
 
    IsOnTime    Month  
 0         1  2025-03  
 1         1  2025-03  
 2         1  2025-03 

In [12]:
# Sanity checks
assert orders["OrderDate"].between("2025-03-16","2025-07-07").all()
assert set(orders["OrderType"].dropna().unique()).issubset({"Subscription","OneTime"})
assert set(orders["DeliverySlot"].dropna().unique()).issubset({"Morning","Evening"})
assert set(orders["OnTime"].dropna().unique()).issubset({"Yes","No"})
assert set(orders["Fulfilled"].dropna().unique()).issubset({"Yes","No"})
assert set(survey["Subscription_Interest"].dropna().unique()).issubset({"Yes","No"})
print("All sanity checks passed ✅")

# Save cleaned files
from pathlib import Path
PROCESSED = Path("data/processed")
PROCESSED.mkdir(parents=True, exist_ok=True)

orders.to_csv(PROCESSED / "orders_clean.csv", index=False)
survey.to_csv(PROCESSED / "survey_clean.csv", index=False)

print("Files saved → data/processed/")


All sanity checks passed ✅
Files saved → data/processed/
